In [1]:
BASE_PATH = '/media/sergio/WD1TB/foto'
BASE_DEST = '/media/sergio/WD1TB/foto_org'

In [3]:
import PIL.Image as Image
import PIL.ExifTags as ExifTags

In [4]:
def date_taken(file):
    # Tags
    ## 306: Datetime
    ## 36867: Datetime original
    ## 36868: Datetime digitalized
    datetime = None
    datetime_original = None
    datetime_digitalized = None
    img = Image.open(file)
    exif = img._getexif()
    if exif:
        datetime = exif.get(306)
        datetime_original = exif.get(36867)
        datetime_digitalized = exif.get(36868)
    img.close()
    return datetime, datetime_original, datetime_digitalized

In [10]:
import pandas as pd
df = pd.DataFrame.from_csv(BASE_PATH+'/movies_list.txt', index_col=None)
df.columns = ['file_path']

In [11]:
df.file_path =  df.file_path.str[1:]

In [12]:
df['datetime'] = [None for i in range(len(df))]
df['original'] = [None for i in range(len(df))]
df['digitalized'] = [None for i in range(len(df))]


In [13]:
for i in range(len(df)):
    try:
    exif = date_taken(BASE_PATH+df.loc[i, 'file_path'])
    df.loc[i,'datetime'] = exif[0]
    df.loc[i, 'original'] = exif[1]
    df.loc[i, 'digitalized'] = exif[2]
  

OSError: cannot identify image file '/media/sergio/WD1TB/foto/recup_dir.109/f73730208.mp4'

In [392]:
df['no_date'] =  df.datetime.isnull() &  df.original.isnull() & df.digitalized.isnull()

In [393]:
df.isnull().sum()

file_path        0
datetime       321
original       363
digitalized    430
no_date          0
dtype: int64

In [394]:
len(df)

15570

In [395]:
df.to_csv(BASE_PATH+'/jpg_org.txt', sep=';', index=False)
#df = pd.DataFrame.from_csv(BASE_PATH+'/jpg_org.txt', sep=';', index_col=None)

In [396]:
df['fname'] = None

In [397]:
df.fname = df[df.original.isnull() == False].original

In [398]:
df.fname.isnull().sum()

363

In [399]:
idx = (df.fname.isnull()==True)  & (df.datetime.isnull()==False)
df.loc[idx].fname = df[idx].datetime

/home/sergio/miniconda3/lib/python3.4/site-packages/pandas/core/generic.py:2005: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [400]:
df.fname.isnull().sum()

363

In [401]:
idx = (df.fname.isnull()==True)  & (df.digitalized.isnull()==False)
df.loc[idx].fname = df[idx].digitalized

In [402]:
df.isnull().sum()

file_path        0
datetime       321
original       363
digitalized    430
no_date          0
fname          363
dtype: int64

In [403]:
df['year'] = df.fname.str[:4]
df['month'] = df.fname.str[5:7]
df['day'] = df.fname.str[8:10]
df['hour'] = df.fname.str[11:13]
df['minute'] = df.fname.str[14:16]
df['second'] = df.fname.str[17:19]

In [404]:
df['fname_dest'] = df.year  + df.month + df.day + '_' + df.hour + df.minute + df.second
df['folder_dest'] = '/' + df.year+'/'+df.month+'/'+df.day

In [405]:
df['file_name'] = df.file_path.apply(lambda x : x[x.rfind('/')+1:])

In [409]:
dfx = df[df.fname.isnull()==False]
dest = 'mv ' + BASE_PATH  + dfx.file_path + ' ' + BASE_DEST + '/' + dfx.year + '/' + dfx.month + '/' + dfx.day


In [412]:
print(dest.iloc[0])

mv /media/sergio/WD1TB/foto/recup_dir.264/f919764296.jpg /media/sergio/WD1TB/foto_org2/2010/09/07


In [413]:
# Cria pastas com base na data.
import os 

folders = BASE_DEST + '/' + dfx.year + '/' + dfx.month + '/' + dfx.day

def mkdir(x):
    if not os.path.exists(x):
        os.makedirs(x)

folders.apply(mkdir);


In [414]:
dest[10:].to_csv('mover_fotos2.txt', index=None)